# Amadeus API to get Flight Offers

- [get-started-with-amadeus-apis](https://developers.amadeus.com/get-started/get-started-with-self-service-apis-335)
- [list-of-amadeus-apis](https://developers.amadeus.com/self-service)
- [flight-offers-api](https://developers.amadeus.com/self-service/category/flights/api-doc/flight-offers-search/api-reference)

In [149]:
import time
import os
import json
import requests
import pandas as pd
import itertools
from datetime import datetime, timedelta
from dateutil.parser import isoparse

In [150]:
airports_df = pd.read_csv("international_airports.csv")

In [151]:
iata_codes = airports_df['iata_code'].tolist()

In [152]:
# Function to get access token
def get_access_token(client_id, client_secret):
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception(f"Failed to get access token: {response.status_code}, {response.text}")

# Retry decorator
def retry_on_failure(retries=5, backoff_factor=1, status_code=429):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(retries):
                result = func(*args, **kwargs)
                if isinstance(result, tuple) and result[1] == status_code:
                    print(f"Rate limit exceeded. Retrying in {backoff_factor * (2 ** attempt)} seconds...")
                    time.sleep(backoff_factor * (2 ** attempt))
                else:
                    return result
            return func(*args, **kwargs)  # Final attempt
        return wrapper
    return decorator

# Function to get flight data
@retry_on_failure(retries=10, backoff_factor=2, status_code=429) # Retry 10 times with exponential backoff
def get_flight_data(access_token, origin_code, destination_code, departure_date):
    url = f"https://test.api.amadeus.com/v2/shopping/flight-offers"
    params = {
        "originLocationCode": origin_code,
        "destinationLocationCode": destination_code,
        "departureDate": departure_date,
        "adults": 1,
        "nonStop": "false",
        "currencyCode": "EUR",
        "max": 50
    }
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers, params=params)
    flights = []
    if response.status_code == 200:
        data = response.json()
        for offer in data['data']:
            for itinerary in offer['itineraries']:
                for segment in itinerary['segments']:
                    origin_airport = airports_df[airports_df['iata_code'] == origin_code].iloc[0]
                    destination_airport = airports_df[airports_df['iata_code'] == destination_code].iloc[0]
                    departure_time = isoparse(segment['departure']['at'])
                    arrival_time = isoparse(segment['arrival']['at'])
                    duration = (arrival_time - departure_time).total_seconds()
                    flights.append({
                        'departure_city': origin_airport['location'],
                        'departure_airport': origin_airport['airport'],
                        'departure_airport_code': origin_code,
                        'departure_country': origin_airport['country'],
                        'departure_country_code': origin_airport['country_code'],
                        'destination_city': destination_airport['location'],
                        'destination_airport': destination_airport['airport'],
                        'destination_airport_code': destination_code,
                        'destination_country': destination_airport['country'],
                        'destination_country_code': destination_airport['country_code'],
                        'departure_date': departure_date,
                        'duration_code': itinerary['duration'],
                        'duration': duration,
                        'carrier_code': segment['carrierCode'],
                        'flight_number': segment['number'],
                        'departure_time': departure_time.isoformat(),
                        'arrival_time': arrival_time.isoformat(),
                        'stops': segment['numberOfStops'],
                        'price': offer['price']['total']
                    })
        return flights, 200
    else:
        print(f"Failed to fetch flight data: {response.status_code}, {response.text}")
        return [], response.status_code

# Function to save state
def save_state(state, filename="state.json"):
    with open(filename, 'w') as f:
        json.dump(state, f)

# Function to load state
def load_state(filename="state.json"):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return None

# Function to save flight data
def save_flight_data(flight_data, filename="flight_data.csv"):
    flight_df = pd.DataFrame(flight_data)
    flight_df.to_csv(filename, index=False)

# Function to load flight data
def load_flight_data(filename="flight_data.csv"):
    try:
        flight_df = pd.read_csv(filename)
        return flight_df.to_dict('records')
    except FileNotFoundError:
        return []

In [153]:
# API Key et Secret pour Amadeus
client_id = "IGnGRFWIONOz3O5GcB97FMh9GFgOYCGG"
client_secret = "f0ao9LtaW4UZiaU8"

# Get access token
access_token = get_access_token(client_id, client_secret)

# Generate combinations of IATA codes for departure and destination airports
iata_combinations = list(itertools.permutations(iata_codes, 2))

# Load previous state if it exists
state = load_state()
if state:
    last_origin_code = state['last_origin_code']
    last_destination_code = state['last_destination_code']
    last_date_str = state['last_date']
    flight_data = load_flight_data()
else:
    last_origin_code = None
    last_destination_code = None
    last_date_str = None
    flight_data = []

# If a date was saved, start from that date
if last_date_str:
    start_date = datetime.strptime(last_date_str, '%Y-%m-%d')
else:
    start_date = datetime(2024, 7, 3)

end_date = datetime(2024, 7, 31)
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days + 1)]

# Iterate over IATA code combinations and dates to fetch data
for origin_code, destination_code in iata_combinations:
    if last_origin_code and last_destination_code and (origin_code != last_origin_code or destination_code != last_destination_code):
        continue

    for date in date_generated:
        if last_date_str and date.strftime('%Y-%m-%d') <= last_date_str:
            continue

        date_str = date.strftime('%Y-%m-%d')
        flights, _ = get_flight_data(access_token, origin_code, destination_code, date_str)
        flight_data.extend(flights)

        # Save state and flight data after each request
        state = {
            'last_origin_code': origin_code,
            'last_destination_code': destination_code,
            'last_date': date_str,
            'flight_data': flight_data
        }
        save_state(state)
        save_flight_data(flight_data)

# Convert flight data to DataFrame
flight_df = pd.DataFrame(flight_data)

# Delete saved state after completing the script
if os.path.exists("state.json"):
    os.remove("state.json")

print("Data saved to 'flight_data.csv'")

Failed to fetch flight data: 429, 
            {
                "errors": [
                    {
                        "code": 38194,
                        "title": "Too many requests",
                        "detail": "The network rate limit is exceeded, please try again later",
                        "status": 429
                    }
                ]
            }
        
Rate limit exceeded. Retrying in 2 seconds...
Failed to fetch flight data: 429, 
            {
                "errors": [
                    {
                        "code": 38194,
                        "title": "Too many requests",
                        "detail": "The network rate limit is exceeded, please try again later",
                        "status": 429
                    }
                ]
            }
        
Rate limit exceeded. Retrying in 2 seconds...
Failed to fetch flight data: 429, 
            {
                "errors": [
                    {
                        "code":

KeyboardInterrupt: 

In [135]:
flight_data[-1]

IndexError: list index out of range